In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve
import scanpy as sc
from scipy.sparse import issparse
import os
import sys

backend_path = os.path.join(os.path.dirname(os.getcwd()), 'Backend')
if backend_path not in sys.path:
    sys.path.append(backend_path)

from process import load_adata_to_cache, get_kosara_data

/home/siyuan/miniconda3/envs/biovis/lib/python3.11/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
/home/siyuan/miniconda3/envs/biovis/lib/python3.11/site-packages/anndata/__init__.py:44: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  return module_get_attr_redirect(attr_name, deprecated_mapping=_DEPRECATED)


In [2]:
load_adata_to_cache(sample_ids=["skin_TXK6Z4X_A1_8um"])
result = get_kosara_data(sample_ids=["skin_TXK6Z4X_A1_8um"], gene_list=["COL1A1", "COL3A1"])

Loaded AnnData for sample skin_TXK6Z4X_A1_8um


In [3]:
result

{'skin_TXK6Z4X_A1_8um': [{'id': 's_008um_00301_00321-1',
   'cell_x': 5350.304622134452,
   'cell_y': 1439.119613458518,
   'cell_type': 'Differentiated_KC',
   'total_expression': 347.0,
   'angles': {'COL1A1': 0.0, 'COL3A1': 0.0},
   'radius': {'COL1A1': 0.0, 'COL3A1': 0.0},
   'ratios': {'COL1A1': 0.0, 'COL3A1': 0.0}},
  {'id': 's_008um_00526_00291-1',
   'cell_x': 5878.579029905326,
   'cell_y': 5033.879577733487,
   'cell_type': 'Differentiated_KC',
   'total_expression': 122.0,
   'angles': {'COL1A1': 41.21366386440382, 'COL3A1': 37.65237839711884},
   'radius': {'COL1A1': 3.5041653504348518, 'COL3A1': 3.0800662908214034},
   'ratios': {'COL1A1': 0.05737704783678055, 'COL3A1': 0.032786883413791656}},
  {'id': 's_008um_00320_00191-1',
   'cell_x': 7435.065976381158,
   'cell_y': 1715.4669790369676,
   'cell_type': 'Differentiated_KC',
   'total_expression': 343.0,
   'angles': {'COL1A1': 0.0, 'COL3A1': 0.0},
   'radius': {'COL1A1': 0.0, 'COL3A1': 0.0},
   'ratios': {'COL1A1': 0.0,